In [1]:
import RPi.GPIO as gpio
import time
import random
from time import sleep
from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets
import cv2
import threading
__speed__ = 0
import ObjectDetector as ObD
from ObjectDetector import loadModel , ROI , DetectLoc

from CNN_ModelMaker import Sequential , Dense , Flatten , ReLU , Softmax

gpio.setmode(gpio.BOARD)
gpio.setwarnings(False)

In [2]:
traffic_sign = ''
__speed__ = 0

In [3]:
cam = cv2.VideoCapture(0)

class CamView :
    
    def __init__(self,detLabel='') :
        self.detLabel = detLabel
    
    def show(self,size=400,Model=None) :
        pTime = 0 
        try :
            while True :
                cTime = time.time()
                fps = 1/(cTime-pTime)
                pTime = cTime
                ret , frame = cam.read()
                frame = cv2.resize(frame,(size,size))
                frame = cv2.flip(frame,-1)
                point = DetectLoc(frame,Model,99.5)
                global traffic_sign 
                if point is not None :
                    #print(f'\r{point}',end="")
                    self.detLabel = point[0]
                    traffic_sign = point[0]
                    frame = cv2.rectangle(frame,
                                          (point[1],point[2]),
                                          (point[3],point[4]),
                                          (255,255,255),
                                          2)
                    frame = cv2.putText(frame,
                                        traffic_sign,
                                        (30,300),
                                        cv2.FONT_HERSHEY_SIMPLEX,.5,
                                        (255,255,255),
                                        1,cv2.LINE_AA)
                else :
                    traffic_sign = ''
#                 if warn or dist <= 30 :
#                     frame = cv2.putText(frame,"WARNING !!!",(20,100),
#                                         cv2.FONT_HERSHEY_SIMPLEX,.75,(0,0,200),
#                                         2,cv2.LINE_AA)
                frame = cv2.putText(frame,
                                    "Speed : " + str(__speed__)+' KMPH',
                                    (30,330),
                                    cv2.FONT_HERSHEY_SIMPLEX,.5,
                                    (255,255,255),
                                    1,cv2.LINE_AA)
                frame = cv2.putText(frame," fps : "+str(int(fps)),(310,50),
                                    cv2.FONT_HERSHEY_SIMPLEX,.5,(255,255,255),
                                    1,cv2.LINE_AA)
                cv2.imshow('Car View',frame)
                cv2.waitKey(1)
        except Exception as e:
            cv2.destroyAllWindows()
            print(e)

In [4]:
class Motor() :
    def __init__ (self,Mot,err=None) :
        
        self.Mot = Mot
        
        for M in Mot :
            for key in M :
                if key != 'P' :
                    gpio.setup(M[key],gpio.OUT)
                    
        self.pwm_F_L = gpio.PWM(Mot[0]['E'],100)
        self.pwm_F_R = gpio.PWM(Mot[1]['E'],100)
        self.pwm_B_L = gpio.PWM(Mot[2]['E'],100)
        self.pwm_B_R = gpio.PWM(Mot[3]['E'],100)
        
        self.pwm_F_L.start(0)
        self.pwm_F_R.start(0)
        self.pwm_B_L.start(0)
        self.pwm_B_R.start(0)
        
        self.__speed__ = 0
        self.__error__ = err
        
        self.__maxSpeed__ = 60
        self.__maxTurn__ = 60
        
    def move(self,speed=0,turn=0,t=.5,lSpeed=None,rSpeed=None) :
        k = self.__maxSpeed__
        tu = self.__maxTurn__
        speed *= k
        turn *= tu
        if ( lSpeed is None ) and ( rSpeed is None ) :
            lSpeed = speed - turn
            rSpeed = speed + turn

            if lSpeed>k : lSpeed = k
            elif lSpeed<-k : lSpeed = -k
            if rSpeed>k : rSpeed = k
            elif rSpeed<-k : rSpeed = -k
        else :
            lSpeed = self.__maxSpeed__ * lSpeed / 100
            rSpeed = self.__maxSpeed__ * rSpeed / 100
        
        #print(lSpeed , rSpeed)
        
        self.pwm_F_L.ChangeDutyCycle(abs(lSpeed))
        self.pwm_F_R.ChangeDutyCycle(abs(rSpeed))
        self.pwm_B_L.ChangeDutyCycle(abs(lSpeed))
        self.pwm_B_R.ChangeDutyCycle(abs(rSpeed))
        
        if lSpeed>0 :
            gpio.output(self.Mot[0]['I'],gpio.HIGH)
            gpio.output(self.Mot[0]['O'],gpio.LOW)
            gpio.output(self.Mot[2]['I'],gpio.HIGH)
            gpio.output(self.Mot[2]['O'],gpio.LOW)
        else :
            gpio.output(self.Mot[0]['O'],gpio.HIGH)
            gpio.output(self.Mot[0]['I'],gpio.LOW)
            gpio.output(self.Mot[2]['O'],gpio.HIGH)
            gpio.output(self.Mot[2]['I'],gpio.LOW)
        if rSpeed>0 :
            gpio.output(self.Mot[1]['I'],gpio.HIGH)
            gpio.output(self.Mot[1]['O'],gpio.LOW)
            gpio.output(self.Mot[3]['I'],gpio.HIGH)
            gpio.output(self.Mot[3]['O'],gpio.LOW)
        else :
            gpio.output(self.Mot[1]['O'],gpio.HIGH)
            gpio.output(self.Mot[1]['I'],gpio.LOW)
            gpio.output(self.Mot[3]['O'],gpio.HIGH)
            gpio.output(self.Mot[3]['I'],gpio.LOW)
        
        sleep(t)
        
    def stop(self,speed=0,t=1) :
        self.pwm_F_L.ChangeDutyCycle(speed)
        self.pwm_F_R.ChangeDutyCycle(speed)
        self.pwm_B_L.ChangeDutyCycle(speed)
        self.pwm_B_R.ChangeDutyCycle(speed)
        self.mySpeed = 0
        sleep(t)
        return

In [5]:
Mot = [ {'P' : 'F_L' , 'I' : 26, 'O' : 24, 'E' : 22},
        {'P' : 'F_R' , 'I' : 40, 'O' : 38, 'E' : 36},
        {'P' : 'B_L' , 'I' : 35, 'O' : 37, 'E' : 33},
        {'P' : 'B_R' , 'I' : 11, 'O' : 13, 'E' : 15}]
car = Motor(Mot)
#gpio.cleanup()

In [6]:
Labels = ['Speed limit (30kmph)' ,
          'Speed limit (60kmph)' ,
          'Stop',
          'Turn right ahead',
          'Turn left ahead']

In [7]:
camera = CamView()

In [8]:
def moveCar() :
    while True :
        Label = traffic_sign
        global __speed__
        if Label.lower() == 'Speed limit (30kmph)'.lower() :
            __speed__ = 30
            car.move(t=.25,lSpeed=__speed__,rSpeed=__speed__)
            car.move(t=.15,lSpeed=-1,rSpeed=__speed__)
        elif Label.lower() == 'Speed limit (60kmph)'.lower() :
            __speed__ = 60
            car.move(lSpeed=__speed__,rSpeed=__speed__)
        elif Label.lower() == 'Turn left ahead'.lower() :
            __speed__ = 40
            car.move(t=.15,lSpeed=__speed__,rSpeed=-10)
            car.move(lSpeed=15,rSpeed=15)
        elif Label.lower() == 'Turn right ahead'.lower() :
            __speed__ = 40
            car.move(t=.15,lSpeed=-10,rSpeed=__speed__)
            car.move(lSpeed=15,rSpeed=15)
        elif Label.lower() == 'Stop'.lower() :
            car.move(0,0)
            __speed__ = 0
        else :
            car.move(0,0)
            __speed__ = 0
            car.move(lSpeed=__speed__,rSpeed=__speed__)

In [9]:
fileName = 'Traffic_Signs_Detection_5.obj' 
model = loadModel(fileName)
model.Summary()

____________________________________________________________
Model Summary
____________________________________________________________
Layers                Input Shape           Output Shape         
____________________________________________________________
Flatten               (32, 32, 3)           3072
____________________________________________________________
Dense                 3072                  6
____________________________________________________________


In [10]:
camera_thread = threading.Thread(target=camera.show,args=(400,model))
camera_thread.start()
car_thread = threading.Thread(target=moveCar)
car_thread.start()


### `Forward`

* car.move(.5,0,.5 ,lSpeed=30,rSpeed=30)
* car.move(.5,0,.15,lSpeed=-5,rSpeed=40)

### `Backward`
* car.move(.5,0, .25 ,lSpeed=-30,rSpeed=-30)
* car.move(.5,0,.05,lSpeed=10,rSpeed=-10)

### `Right Turn`
* car.move(.5,0,.4) # Forward Movement
* car.move(.5,-0.5,.25) # Right Turn
* car.move(.5,0,.3) # Forward Movement
* car.move(.5,-0.5,.3) # Right Turn
* car.move(.5,0,.25) # Forward Movement
* car.move(.5,-0.5,.1) # Right Turn

### `Left Turn`
* 